# Naive Bayes dengan Data Kategorikal

Pada percobaan kedua ini, kita akan menggunakan data riil untuk melakukan klasifikasi dengan Naive Bayes. Data yang digunakan adalah **adult.csv**. 

## Inspeksi Data

Pada tahap ini kita akan melakukan loading data dan inspeksi data. Hal ini dilakukan untuk mengetahui apakah kita perlu melakukan proses pendahuluan sebelum melakukan training

In [21]:
import numpy as np
import pandas as pd

# Load data Excel ke Data Frame
df = pd.read_csv('dataset/adult.csv')

# Cek data
df.head()

,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
0,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
1,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
2,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
3,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
4,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K


### Menyempurnakan tabel dengan menambahkan nama kolom

In [22]:
# membuat column
df.columns = ['age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital_status', 'occupation', 'relationship',
             'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'income']

df.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
1,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
2,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
3,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
4,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K


### Cek Kategorikal Variabel nya

In [18]:
## Cek Kategorial Variabel
categorical = [var for var in df.columns if df[var].dtype=='O']

print('There are {} categorical variables\n'.format(len(categorical)))

print('The categorical variables are :\n\n', categorical)

There are 9 categorical variables

The categorical variables are :

 ['workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'native_country', 'income']


In [19]:
# View Categorical Var
df[categorical].head()

,workclass,education,marital_status,occupation,relationship,race,sex,native_country,income
0,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States,<=50K
1,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,United-States,<=50K
2,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,United-States,<=50K
3,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,Cuba,<=50K
4,Private,Masters,Married-civ-spouse,Exec-managerial,Wife,White,Female,United-States,<=50K


In [23]:
# cek missing value pada kategorikal

df[categorical].isnull().sum()

workclass         0
education         0
marital_status    0
occupation        0
relationship      0
race              0
sex               0
native_country    0
income            0
dtype: int64

Berdasarkan pengecekan data, terdapat data bernilai kategorial pada fitur (variabel) ***['workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'native_country', 'income']***. Kita harus melakukan encoding terhadap nilai dari fitur tersebut. Meskipun secara sekilas nilai kategorial menunjukkan tendesi ke tipe data nominal, namun kita tidak akan menggunakan tendik encoding One Hot Encoder atau Ordinal Encoder. Naive Bayes berkerja berdasarkan prinsip probabilitas berkelompok (_join probability_). 

Hal ini membuat merepresentasikan nilai variabel dalam bentuk encoding sebetulnya tidak terlalu penting. Encoding diperlukan hanya untuk menamai ulang nilai kategori dalam bentuk angka dan kebutuhan library scikit-learn yang menggunakan angka sebagai parameter input.

Percobaan kali ini merupakan penerapan klasik algoritma Naive Bayes. Perhatikan kembali contoh intuisi perhitungan Naive Bayes pada modul Jobsheet 3. Meskipun nilai asli dari fitur tidak diketahui, kita masih dapat melakukan proses klasifikasi dengan Naive Bayes.

## Tahap Persiapan

Pada tahap ini kita akan melakukan beberapa hal, yaitu,

1. Encoding nilai kategorikal untuk kebutuhan training
2. Memisahkan fitur dan label
3. Split data training dan testing

### Encoding

In [24]:
# Encoding
# Fungsi encoding yang akan digunakan adalah LabelEncoder
# Hal ini karena kita hanya mengganti nilai variabel dari nama berupa string menjadi angka. Sama halnya dengan label

from sklearn.preprocessing import LabelEncoder

# Inisiasi label encoder
encode = LabelEncoder()

# Terpakan label encoder
df['workclass'] = encode.fit_transform(df['workclass'])
df['education'] = encode.fit_transform(df['education'])
df['marital_status'] = encode.fit_transform(df['marital_status'])
df['occupation'] = encode.fit_transform(df['occupation'])
df['relationship'] = encode.fit_transform(df['relationship'])
df['race'] = encode.fit_transform(df['race'])
df['sex'] = encode.fit_transform(df['sex'])
df['native_country'] = encode.fit_transform(df['native_country'])
df['income'] = encode.fit_transform(df['income'])

# Cek hasil
df.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,50,6,83311,9,13,2,4,0,4,1,0,0,13,39,0
1,38,4,215646,11,9,0,6,1,4,1,0,0,40,39,0
2,53,4,234721,1,7,2,6,0,2,1,0,0,40,39,0
3,28,4,338409,9,13,2,10,5,2,0,0,0,40,5,0
4,37,4,284582,12,14,2,4,5,4,0,0,0,40,39,0


### Memisahkan Fitur dan Label

In [25]:
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

### Split Data Training dan Testing

In [26]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

## Training dan Evaluasi Model

In [27]:
# Kita akan menggunakan CategoricalNB untuk kasus ini
from sklearn.naive_bayes import CategoricalNB
from sklearn.metrics import accuracy_score

# Inisasi CategoricalNB
cnb = CategoricalNB()

# Fit model
# Label y harus dalam bentu 1D atau (n_samples,)
cnb.fit(X_train, y_train)

# Prediksi dengan data training
y_train_pred = cnb.predict(X_train)

# Evaluasi akurasi training
acc_train = accuracy_score(y_train, y_train_pred)

# Prediksi test data
y_test_pred = cnb.predict(X_test)

# Evaluasi model dengan metric akurasi
acc_test = accuracy_score(y_test, y_test_pred)

# Print hasil evaluasi
print(f'Hasil akurasi data train: {acc_train}')
print(f'Hasil akurasi data test: {acc_test}')

Hasil akurasi data train: 0.8876305282555282
Hasil akurasi data test: 0.8624078624078624
